# Detecting Dwarf Galaxies: The Performance & Limitations of Various Density Estimators

### Nicolas Garavito, Jennifer Kadowaki, Matt Kirby, Ben Lew, Ragadeepika Pucha

# Plan - 

- Different Density Detections on Known Dwarfs
- Comparison
- Fake Data 
- M_v VS r_h plot